<center><div style="font-size: 50px"> <b>Put Title Here</b> </center>
<br><br><br>

<center><div style="font-size: 30px"> <b>Introduction</b> </center>

The U.S. recently underwent a presidential election for the year of 2020. In one of the most highly contested and controversial elections of the nation, Joe Biden, the former Vice President under Barack Obama, defeated the incumbent president, Donald Trump. The election was surrounded in controversy, both in terms of the candidates who ran and the election outcome.

Our group chose to analyze data regarding voter turnout rates for the 2020 Presidential Election, and learn a little bit more about what happened in this election. Voter turnout is the percentage of eligible voters who cast a ballot in an election. Eligibility varies by country, and the voting-eligible population should not be confused with the total adult population. Age and citizenship status are often among the criteria used to determine eligibility, but some countries further restrict eligibility based on sex, race, or religion.

The right to vote is one of our hallmark rights, and every U.S. citizen should exercise this right. Our aim in analyzing this data is to share with our viewers a more comprehensive, analytical viewpoint of who voted this election. We hope to shed some light on what went right and wrong this election, and potentially expose some avenues on how we can make it easier to vote throughout the nation. We got our dataset from the website kaggle.com, the famous data science and machine learning hub, and it was provided via data.world by the government.

<br><br><br><br>




<center><div style="font-size: 30px"> <b>Data Collection</b> </center>

The first thing we need to do is obtain the election data at https://www.kaggle.com/imoore/2020-us-general-election-turnout-rates?select=2020+November+General+Election+-+Turnout+Rates.csv.  We then read it into a dataframe using pandas, which allows us to perform operations on the rows and columns of data.  There are originally 15 columns in the election data, but we will filter some out and keep the most important ones.  Additionally, it is neccessary to change the format of some columns to a numeric type, such as total ballots counted, VEP turnout rate, and % non-citizen).  

In [30]:
## data collection
import pandas as pd 
import numpy as np

data = pd.read_csv('election_data.csv')

#deleting columns with NaN values

del data['Source']
del data['Official/Unofficial']
del data['Vote for Highest Office (President)']
del data['Overseas Eligible']

numbers = ['Total Ballots Counted (Estimate)', 'Voting-Eligible Population (VEP)', 'Voting-Age Population (VAP)','Prison', 'Probation', 'Parole', 'Total Ineligible Felon']

floaters = ['VEP Turnout Rate', '% Non-citizen']

#converting strings into proper number format
for c in numbers:
    data[c] = data[c].str.replace(',','')

for c in floaters:
    data[c] = data[c].str.replace('%','')

data[numbers] = data[numbers].astype('int')
data[floaters] = data[floaters].astype('float')

#creating a column for % citizen
data['% Citizen'] = 100 - data['% Non-citizen']

#regional column
west = ['Wyoming','Hawaii','Alaska','California','Nevada','Utah','Colorado','Washington','Oregon','Montana','Idaho']

midwest = ['North Dakota','Minnesota','South Dakota','Wisconsin','Iowa','Illinois','Nebraska','Kansas','Missouri','Ohio','Indiana','Michigan']

northeast = ['Maine','New York','New Jersey','New Hampshire','Massachusetts','Vermont','Rhode Island', 'Connecticut','Pennsylvania']

southeast = ['Maryland','District of Columbia','Delaware','West Virginia','Virginia','Kentucky','Tennessee','North Carolina','South Carolina','Georgia','Alabama','Louisiana','Arkansas','Mississippi','Florida']

southwest = ['Oklahoma','New Mexico','Arizona','Texas']

data['Region'] = data.State.apply(lambda x:
    'W' if x in west else ('MW' if x in midwest else ('NE' if x in northeast else ('SE' if x in southeast else( 'SW' if x in southwest else '--')))) ) 


#reordering columns
data = data[['State','Region','State Abv','Total Ballots Counted (Estimate)', 'VEP Turnout Rate','Voting-Eligible Population (VEP)', 'Voting-Age Population (VAP)','% Citizen','% Non-citizen','Prison', 'Probation', 'Parole', 'Total Ineligible Felon']]


data.head()

,State,Region,State Abv,Total Ballots Counted (Estimate),VEP Turnout Rate,Voting-Eligible Population (VEP),Voting-Age Population (VAP),% Citizen,% Non-citizen,Prison,Probation,Parole,Total Ineligible Felon
0,United States,--,NaN,158835004,66.4,239247182,257605088,92.2,7.8,1461074,1962811,616440,3294457
1,Alabama,SE,AL,2306587,62.6,3683055,3837540,97.7,2.3,25898,50997,10266,67782
2,Alaska,W,AK,367000,69.8,525568,551117,96.6,3.4,4293,2074,1348,6927
3,Arizona,SW,AZ,3400000,65.5,5189000,5798473,91.1,8.9,38520,76844,7536,93699
4,Arkansas,SE,AR,1212030,55.5,2182375,2331171,96.4,3.6,17510,36719,24698,64974


<br><br><br><br>
<center><div style="font-size: 30px"> <b>Data Processing</b> </center>

In [38]:
#code for data processing
import pandas as pd
import numpy as np

def replacer(x):
    return x.fillna(0)

electional = pd.read_csv('county_statistics.csv')

electional = electional[['state','percentage20_Donald_Trump',
'percentage20_Joe_Biden','total_votes20','votes20_Donald_Trump','votes20_Joe_Biden','cases','deaths',]]

##COVID CASES NOVEMBER 1st
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

states_holder = []
d = electional.loc[electional['state'] == 'SC']
for s in states:
    d = electional.loc[electional['state'] == s]
    d = pd.DataFrame(d)
    states_holder.append(d)

for i in range(50):
    states_holder[i] = replacer(states_holder[i])    
    states_holder[i] = states_holder[i].groupby(['state'], as_index = False).sum()
    states_holder[i] = states_holder[i].rename(columns = {'state' : 'State Abv'})

electional = pd.concat(states_holder)
del electional['state']

final_data = pd.merge(electional, data, on = 'State Abv', how = 'left').dropna()

final_data.head()

,State Abv,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,State,Region,Total Ballots Counted (Estimate),VEP Turnout Rate,Voting-Eligible Population (VEP),Voting-Age Population (VAP),% Citizen,% Non-citizen,Prison,Probation,Parole,Total Ineligible Felon
0,AL,43.410,22.924,2309900.0,1434159.0,843473.0,193985.0,2973.0,Alabama,SE,2306587,62.6,3683055,3837540,97.7,2.3,25898,50997,10266,67782
1,AK,22.411,15.939,260983.0,148624.0,102080.0,16705.0,83.0,Alaska,W,367000,69.8,525568,551117,96.6,3.4,4293,2074,1348,6927
2,AZ,8.227,6.547,3366867.0,1651812.0,1663447.0,247473.0,5979.0,Arizona,SW,3400000,65.5,5189000,5798473,91.1,8.9,38520,76844,7536,93699
3,AR,51.710,21.084,1209372.0,757052.0,418051.0,113057.0,1958.0,Arkansas,SE,1212030,55.5,2182375,2331171,96.4,3.6,17510,36719,24698,64974
4,CA,25.089,31.595,16092730.0,5416035.0,10339137.0,935878.0,17671.0,California,W,16800000,64.7,25962648,30783255,85.0,15.0,104730,0,102586,207316


<br><br><br><br>
<center><div style="font-size: 30px"> <b>Data Analysis/Visualization</b> </center>

In [3]:
#code for data analyis/visualization

<br><br><br><br>
<center><div style="font-size: 30px"> <b>Hypothesis Testing/ML</b> </center>

In [4]:
#code for hypothesis testing/ML

<br><br><br><br>
<center><div style="font-size: 30px"> <b>Analysis</b> </center>

In [5]:
#code for analysis